<a href="https://colab.research.google.com/github/victorvalente/HarmonicOscillatorPSO/blob/main/HarmonicOscillatorPSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from enum import Enum, auto
import os
import glob
import json
from collections import defaultdict, deque
import time
from scipy.fft import rfft, rfftfreq # For frequency domain features
import traceback # For detailed error printing

# --- Configuration & Enums ---

class SensorType(Enum):
    ACCELEROMETER = auto()
    PIR = auto()
    RSSI = auto()
    VIDEO = auto()

class ActivityType(Enum):
    AMBULATION = auto()
    POSTURE = auto()
    TRANSITION = auto()

class Location(Enum):
    BATH = auto()
    BED1 = auto()
    BED2 = auto()
    HALL = auto()
    KITCHEN = auto()
    LIVING = auto()
    STAIRS = auto()
    STUDY = auto()
    TOILET = auto()
    UNKNOWN = auto()

SPHERE_ACTIVITIES = {
    'a_ascend': ActivityType.AMBULATION, 'a_descend': ActivityType.AMBULATION,
    'a_jump': ActivityType.AMBULATION, 'a_loadwalk': ActivityType.AMBULATION,
    'a_walk': ActivityType.AMBULATION, 'p_bent': ActivityType.POSTURE,
    'p_kneel': ActivityType.POSTURE, 'p_lie': ActivityType.POSTURE,
    'p_sit': ActivityType.POSTURE, 'p_squat': ActivityType.POSTURE,
    'p_stand': ActivityType.POSTURE, 't_bend': ActivityType.TRANSITION,
    't_kneel_stand': ActivityType.TRANSITION, 't_lie_sit': ActivityType.TRANSITION,
    't_sit_lie': ActivityType.TRANSITION, 't_sit_stand': ActivityType.TRANSITION,
    't_stand_kneel': ActivityType.TRANSITION, 't_stand_sit': ActivityType.TRANSITION,
    't_straighten': ActivityType.TRANSITION, 't_turn': ActivityType.TRANSITION
}
SPHERE_ACTIVITY_NAMES = list(SPHERE_ACTIVITIES.keys())

SPHERE_LOCATIONS = ['bath', 'bed1', 'bed2', 'hall', 'kitchen', 'living', 'stairs', 'study', 'toilet']
SPHERE_LOCATION_ENUM_MAP = {name: Location[name.upper()] for name in SPHERE_LOCATIONS}
SPHERE_LOCATION_ENUM_MAP['UNKNOWN'] = Location.UNKNOWN

# --- Data Handling Utilities (Revised v4 - Step 1) ---

def load_sphere_data(data_path, subject_id, data_type):
    """
    Loads specific sensor data or annotations for a subject,
    adapted for potential SPHERE challenge structures (v4 - handles PIR start/end).
    """
    print(f"Attempting to load {data_type} for subject {subject_id} from base path: {data_path}")
    file_path = None # Used for logging the primary file path loaded
    df = None      # DataFrame to be loaded

    # --- Path and Filename Logic ---
    data_type_lower = data_type.lower()

    # Determine base subject folder (try train, test, then base)
    subject_folder_options = [
        os.path.join(data_path, 'train', subject_id),
        os.path.join(data_path, 'test', subject_id),
        os.path.join(data_path, subject_id)
    ]
    subject_base_path = None
    for folder_option in subject_folder_options:
        if os.path.isdir(folder_option):
            subject_base_path = folder_option
            print(f"  Using subject data folder: {subject_base_path}")
            break

    if subject_base_path is None:
         if data_type_lower == 'annotations':
              print("  Subject folder not found, checking for annotations in base/train paths...")
              pass
         else:
              print(f"Error: Could not find subject folder for {subject_id} in train/, test/, or base directory relative to {data_path}.")
              return None

    # --- Specific Loading Logic per Type ---
    if data_type_lower == 'annotations':
        annotation_files = []
        if subject_base_path:
             annotation_pattern = os.path.join(subject_base_path, 'annotations_*.csv')
             annotation_files = sorted(glob.glob(annotation_pattern))
        if not annotation_files:
             print(f"  Annotations not found in subject folder, checking other locations...")
             possible_paths = [ os.path.join(data_path,'train',f'{subject_id}_annotations.csv'), os.path.join(data_path,'train','annotations.csv'),
                                os.path.join(data_path,'annotations',f'{subject_id}.csv'), os.path.join(data_path,'annotations.csv') ]
             for p in possible_paths:
                 if os.path.exists(p): annotation_files = [p]; print(f"  Found potential annotation file at: {p}"); break
        if not annotation_files: print(f"Error: No annotation files found for subject {subject_id}."); return None

        print(f"  Found annotation files: { [os.path.basename(f) for f in annotation_files] }")
        file_path = annotation_files[0]
        all_annotations_df = []
        for f in annotation_files:
             try: df_part = pd.read_csv(f); all_annotations_df.append(df_part); print(f"    Successfully loaded {os.path.basename(f)}")
             except Exception as e: print(f"Warning: Could not load or read annotation file {f}: {e}")
        if not all_annotations_df: print(f"Error: Failed to load any valid data from found annotation files."); return None
        df = pd.concat(all_annotations_df, ignore_index=True); print(f"  Concatenated {len(annotation_files)} annotation files.")
    else:
        if subject_base_path is None: return None
        filename_map = {'accelerometer':['acceleration.csv','acc*.csv'], 'pir':['pir.csv'], 'rssi':['rssi*.csv'], 'video_features':['video_features*.csv']}
        patterns = filename_map.get(data_type_lower)
        if not patterns: print(f"Error: Unknown data_type '{data_type_lower}'."); return None
        found_files = []
        for pattern in patterns:
             matching_files = glob.glob(os.path.join(subject_base_path, pattern))
             if matching_files: found_files.extend(matching_files)
        if not found_files: print(f"Warning: No file matching '{patterns}' for {data_type_lower} in {subject_base_path}"); return None
        primary_target = os.path.join(subject_base_path, patterns[0])
        file_path = primary_target if primary_target in found_files else sorted(found_files)[0]
        if file_path != primary_target: print(f"  Note: Using first matched file: {os.path.basename(file_path)}")
        print(f"  Loading data from: {file_path}")
        try: df = pd.read_csv(file_path)
        except Exception as e: print(f"Error loading sensor data file {file_path}: {e}"); traceback.print_exc(); return None

    # --- Common Processing (Timestamping, Indexing) ---
    if df is None: print(f"Error: DataFrame is None for {data_type_lower}."); return None
    try:
        timestamp_cols_to_try = ['t', 'timestamp', 'time']
        timestamp_col = None
        for col in timestamp_cols_to_try:
            if col in df.columns: timestamp_col = col; print(f"  Found timestamp column: '{timestamp_col}'"); break

        if timestamp_col is None and (data_type_lower == 'annotations' or data_type_lower == 'pir'):
            if 'start' in df.columns and 'end' in df.columns:
                 timestamp_col = 'start'; print(f"  Using 'start' column as timestamp index for {data_type_lower}.")
                 df['start'] = pd.to_datetime(df['start'], unit='s'); df['end'] = pd.to_datetime(df['end'], unit='s') # <<< VERIFY UNIT
                 df = df.set_index('start').sort_index(); print(f"  ...Processed {len(df)} {data_type_lower} rows."); return df
            else: print(f"Error: {data_type_lower.capitalize()} file lacks 'start'/'end' and standard timestamp columns."); return None
        elif timestamp_col is None: print(f"Error: Could not find suitable timestamp column {timestamp_cols_to_try} in {file_path}"); return None

        if timestamp_col != 'timestamp': df = df.rename(columns={timestamp_col: 'timestamp'})
        print(f"  Converting timestamp column (assuming units are seconds)..."); df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s') # <<< VERIFY UNIT
        df = df.set_index('timestamp').sort_index(); print(f"  ...Loaded {len(df)} rows."); return df
    except Exception as e: print(f"Error processing DataFrame for {file_path if file_path else data_type}: {e}"); traceback.print_exc(); return None


def synchronize_data(data_streams, resample_freq='50ms'):
    """Synchronizes multiple data streams via resampling and forward-fill."""
    print(f"Synchronizing data streams to {resample_freq} frequency...")
    combined_df = None; processed_streams = {}
    for sensor_type, df in data_streams.items():
        if df is None or df.empty: print(f"  Skipping empty stream: {sensor_type.name}"); continue
        print(f"  Processing {sensor_type.name}...")
        if not isinstance(df.index, pd.DatetimeIndex): print(f"Error: DataFrame for {sensor_type.name} lacks DatetimeIndex."); continue
        try:
            if sensor_type == SensorType.PIR:
                if 'end' in df.columns:
                     min_time=df.index.min(); max_time=df['end'].max()
                     if pd.isna(min_time) or pd.isna(max_time): print(f"Warning: Invalid min/max time for PIR {sensor_type.name}. Skipping."); continue
                     target_index=pd.date_range(start=min_time,end=max_time,freq=resample_freq)
                     resampled_df_pir=pd.DataFrame(0,index=target_index,columns=SPHERE_LOCATIONS)
                     for start, row in df.iterrows():
                          end=row['end']; active_indices=target_index[(target_index>=start)&(target_index<end)]
                          if 'name' in row.index and row['name'] in SPHERE_LOCATIONS: resampled_df_pir.loc[active_indices,row['name']]=1
                     resampled_df=resampled_df_pir.fillna(0)
                else: print(f"Warning: Cannot resample PIR {sensor_type.name} without 'end' column. Skipping."); continue
            else: resampled_df = df.resample(resample_freq).mean()
            resampled_df.columns=[f"{col}_{sensor_type.name}" for col in resampled_df.columns]; processed_streams[sensor_type]=resampled_df; print(f"    Resampled {sensor_type.name} to {len(resampled_df)} rows.")
        except Exception as e: print(f"Error resampling {sensor_type.name}: {e}"); traceback.print_exc()
    print("  Merging streams...")
    first_stream = True
    for sensor_type, resampled_df in processed_streams.items():
        if first_stream: combined_df=resampled_df; first_stream=False
        else: combined_df=pd.merge(combined_df,resampled_df,left_index=True,right_index=True,how='outer'); print(f"    Merged {sensor_type.name}, shape now: {combined_df.shape}")
    if combined_df is not None and not combined_df.empty:
        print("  Forward-filling missing values..."); combined_df=combined_df.ffill(); combined_df=combined_df.bfill(); combined_df=combined_df.fillna(0); print(f"  Final synchronized shape: {combined_df.shape}")
    elif combined_df is not None and combined_df.empty: print("Warning: Combined DataFrame is empty after merging.")
    else: print("Error: No data streams could be processed and merged."); return None
    return combined_df

def create_time_windows(sync_data, window_size_sec, overlap_sec):
    """Creates time-windowed chunks from synchronized data."""
    if sync_data is None or sync_data.empty: print("Error: Cannot create windows from empty sync data."); return
    print(f"Creating time windows (size: {window_size_sec}s, overlap: {overlap_sec}s)...")
    window_delta=pd.Timedelta(seconds=window_size_sec); step_delta=pd.Timedelta(seconds=window_size_sec-overlap_sec)
    if step_delta.total_seconds()<=0: print("Error: Overlap >= window size."); return
    start_time=sync_data.index.min(); end_time=sync_data.index.max(); current_time=start_time; window_count=0
    while current_time + window_delta <= end_time:
        window_end=current_time+window_delta; window_df=sync_data[(sync_data.index>=current_time)&(sync_data.index<window_end)]
        if window_df.empty: current_time+=step_delta; continue
        window_chunks={}
        for sensor_type in SensorType:
            sensor_suffix=f"_{sensor_type.name}"; sensor_cols_with_suffix=[col for col in window_df.columns if col.endswith(sensor_suffix)]
            if sensor_cols_with_suffix:
                sensor_chunk=window_df[sensor_cols_with_suffix].copy(); sensor_chunk.columns=[col.replace(sensor_suffix,'') for col in sensor_cols_with_suffix]
                if sensor_type == SensorType.PIR:
                    for loc in SPHERE_LOCATIONS:
                        if loc not in sensor_chunk.columns: sensor_chunk[loc]=0
                    if all(loc in sensor_chunk.columns for loc in SPHERE_LOCATIONS): window_chunks[sensor_type]=sensor_chunk[SPHERE_LOCATIONS]
                    else: print(f"Warning: PIR chunk missing location columns at {window_end_time}"); window_chunks[sensor_type]=sensor_chunk
                else: window_chunks[sensor_type]=sensor_chunk
        yield {'timestamp':window_end,'data':window_chunks}; window_count+=1; current_time+=step_delta
    print(f"Generated {window_count} windows.")


# --- Agent Class ---
class Agent:
    """An agent with unique sensory capabilities analyzing sensor data."""
    def __init__(self, agent_id, sensor_type, feature_extractors=None, learning_rate=0.1, history_len=10):
        self.id=agent_id; self.sensor_type=sensor_type; self.learning_rate=learning_rate; self.history=deque(maxlen=history_len)
        self.confidence=0.5; self.activity_accuracy={act:0.5 for act in SPHERE_ACTIVITY_NAMES}; self.knowledge={act:1.0/len(SPHERE_ACTIVITY_NAMES) for act in SPHERE_ACTIVITY_NAMES}; self.activity_specialization={act:0.5 for act in SPHERE_ACTIVITY_NAMES}; self.transition_model={}
        self.feature_extractors=feature_extractors or self._default_feature_extractors(); self.last_timestamp=None

    def _default_feature_extractors(self):
        global RESAMPLE_FREQ
        try: sampling_rate=1.0/pd.Timedelta(RESAMPLE_FREQ).total_seconds()
        except NameError: sampling_rate=20.0; print(f"Warning: Assuming sampling rate {sampling_rate} Hz.")
        if self.sensor_type==SensorType.ACCELEROMETER:
            return {'mean':lambda x:np.nanmean(x,axis=0) if x.shape[0]>0 else np.zeros(3),'std':lambda x:np.nanstd(x,axis=0) if x.shape[0]>0 else np.zeros(3),'max':lambda x:np.nanmax(x,axis=0) if x.shape[0]>0 else np.zeros(3),'min':lambda x:np.nanmin(x,axis=0) if x.shape[0]>0 else np.zeros(3),
                    'range':lambda x:np.ptp(x[~np.isnan(x).any(axis=1)],axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>0 else np.zeros(3),'median':lambda x:np.nanmedian(x,axis=0) if x.shape[0]>0 else np.zeros(3),
                    'energy':lambda x:np.sum(np.square(x[~np.isnan(x).any(axis=1)]),axis=0)/x.shape[0] if x.shape[0]>0 else np.zeros(3),
                    'iqr':lambda x:np.subtract(*np.nanpercentile(x[~np.isnan(x).any(axis=1)],[75,25],axis=0)) if x.shape[0]>0 and np.sum(~np.isnan(x))>1 else np.zeros(3),
                    'fft_energy':lambda x:self._calculate_fft_energy(x,sampling_rate) if x.shape[0]>1 else np.zeros(3),'fft_peak_freq':lambda x:self._calculate_fft_peak_freq(x,sampling_rate) if x.shape[0]>1 else np.zeros(3),}
        elif self.sensor_type==SensorType.PIR:
             return {'sum':lambda x:np.nansum(x.values,axis=0) if x.shape[0]>0 else np.zeros(len(SPHERE_LOCATIONS)),'any':lambda x:np.nanmax(x.values,axis=0).astype(int) if x.shape[0]>0 else np.zeros(len(SPHERE_LOCATIONS)),
                     'count':lambda x:np.nansum(x.values>0,axis=0) if x.shape[0]>0 else np.zeros(len(SPHERE_LOCATIONS)),}
        elif self.sensor_type==SensorType.RSSI:
             num_aps_init=4; return {'mean':lambda x:np.nanmean(x,axis=0) if x.shape[0]>0 else np.full(num_aps_init,-100.0),'std':lambda x:np.nanstd(x,axis=0) if x.shape[0]>0 else np.zeros(num_aps_init),
                                     'max':lambda x:np.nanmax(x,axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>0 else np.full(num_aps_init,-100.0),'min':lambda x:np.nanmin(x,axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>0 else np.full(num_aps_init,-100.0),
                                     'diff':lambda x:np.nanmax(x,axis=0)-np.nanmin(x,axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>1 else np.zeros(num_aps_init),'stability':lambda x:1.0/(1.0+np.nanstd(x,axis=0)) if x.shape[0]>0 else np.zeros(num_aps_init)}
        elif self.sensor_type==SensorType.VIDEO:
             return {'center_mean':lambda x:np.nanmean(x[['centre_2d_x','centre_2d_y']].values,axis=0) if x.shape[0]>0 else np.zeros(2),'center_std':lambda x:np.nanstd(x[['centre_2d_x','centre_2d_y']].values,axis=0) if x.shape[0]>0 else np.zeros(2),
                     'area_mean':lambda x:self._calculate_area(x,'mean') if x.shape[0]>0 else np.zeros(1),'area_std':lambda x:self._calculate_area(x,'std') if x.shape[0]>0 else np.zeros(1),
                     'aspect_ratio_mean':lambda x:self._calculate_aspect_ratio(x,'mean') if x.shape[0]>0 else np.zeros(1),'movement_mean':lambda x:self._calculate_movement(x,'mean') if x.shape[0]>1 else np.zeros(1),
                     'height_mean':lambda x:self._calculate_height(x,'mean') if x.shape[0]>0 else np.zeros(1),'y_center_mean':lambda x:np.nanmean(x['centre_2d_y'].values) if x.shape[0]>0 else np.zeros(1),}
        else: return {'identity':lambda x:x}

    def _calculate_fft_energy(self, data, sampling_rate):
        energies=[]; n_samples=data.shape[0];
        if n_samples==0: return np.zeros(data.shape[1])
        for i in range(data.shape[1]):
            col_data=data[:,i]; valid_data=col_data[~np.isnan(col_data)]; n_valid=len(valid_data)
            if n_valid<2: energies.append(0); continue
            fft_values=rfft(valid_data); fft_power=np.abs(fft_values)**2/n_valid; energies.append(np.sum(fft_power))
        return np.array(energies)
    def _calculate_fft_peak_freq(self, data, sampling_rate):
        peak_freqs=[]; n_samples=data.shape[0];
        if n_samples==0: return np.zeros(data.shape[1])
        for i in range(data.shape[1]):
            col_data=data[:,i]; valid_data=col_data[~np.isnan(col_data)]; n_valid=len(valid_data)
            if n_valid<2 or sampling_rate<=0: peak_freqs.append(0); continue
            freqs=rfftfreq(n_valid,1.0/sampling_rate); fft_values=rfft(valid_data); fft_power=np.abs(fft_values)**2; valid_freq_indices=np.arange(len(freqs))
            if len(fft_power)>1: peak_index_rel=np.argmax(fft_power[1:]); peak_index_abs=valid_freq_indices[1:][peak_index_rel]; peak_freqs.append(freqs[peak_index_abs])
            else: peak_freqs.append(0)
        return np.array(peak_freqs)
    def _calculate_area(self, x, stat='mean'):
        req=['bb_2d_br_x','bb_2d_br_y','bb_2d_tl_x','bb_2d_tl_y'];
        if not all(c in x.columns for c in req): return np.zeros(1)
        w=x['bb_2d_br_x']-x['bb_2d_tl_x']; h=x['bb_2d_br_y']-x['bb_2d_tl_y']; a=w*h;
        if a.isnull().all(): return np.zeros(1)
        if stat=='mean': return np.array([np.nanmean(a)])
        if stat=='std': return np.array([np.nanstd(a)])
        return np.zeros(1)
    def _calculate_aspect_ratio(self, x, stat='mean'):
        req=['bb_2d_br_x','bb_2d_br_y','bb_2d_tl_x','bb_2d_tl_y'];
        if not all(c in x.columns for c in req): return np.zeros(1)
        w=x['bb_2d_br_x']-x['bb_2d_tl_x']; h=x['bb_2d_br_y']-x['bb_2d_tl_y']; v=(h!=0)&(~h.isnull())&(~w.isnull());
        if not v.any(): return np.zeros(1)
        r=pd.Series(np.nan,index=x.index); r.loc[v]=w.loc[v]/h.loc[v];
        if r.isnull().all(): return np.zeros(1)
        if stat=='mean': return np.array([np.nanmean(r)])
        if stat=='std': return np.array([np.nanstd(r)])
        return np.zeros(1)
    def _calculate_movement(self, x, stat='mean'):
        req=['centre_2d_x','centre_2d_y'];
        if not all(c in x.columns for c in req) or x.shape[0]<2: return np.zeros(1)
        cx=x['centre_2d_x']; cy=x['centre_2d_y']; dx=cx-cx.shift(1); dy=cy-cy.shift(1); m=np.sqrt(dx**2+dy**2);
        if m.isnull().all(): return np.zeros(1)
        if stat=='mean': return np.array([np.nanmean(m)])
        if stat=='std': return np.array([np.nanstd(m)])
        return np.zeros(1)
    def _calculate_height(self, x, stat='mean'):
        req=['bb_2d_br_y','bb_2d_tl_y'];
        if not all(c in x.columns for c in req): return np.zeros(1)
        h=x['bb_2d_br_y']-x['bb_2d_tl_y'];
        if h.isnull().all(): return np.zeros(1)
        if stat=='mean': return np.array([np.nanmean(h)])
        if stat=='std': return np.array([np.nanstd(h)])
        return np.zeros(1)

    def process_reading(self, timestamp, data_chunk, ground_truth_activity=None):
        self.last_timestamp=timestamp
        if data_chunk is None or (isinstance(data_chunk,pd.DataFrame) and data_chunk.empty):
             d_s={act:1.0/len(SPHERE_ACTIVITY_NAMES) for act in SPHERE_ACTIVITY_NAMES}; return {'agent_id':self.id,'timestamp':timestamp,'activity_scores':d_s,'confidence':0.1,'prediction':None,'features':None,'error':'Missing data'}
        features=self._extract_features(data_chunk)
        if not features:
             d_s={act:1.0/len(SPHERE_ACTIVITY_NAMES) for act in SPHERE_ACTIVITY_NAMES}; return {'agent_id':self.id,'timestamp':timestamp,'activity_scores':d_s,'confidence':0.1,'prediction':None,'features':features,'error':'Feature extraction failed'}
        activity_scores=self._determine_activity_scores(features)
        if not activity_scores: pred_act=None
        else: scores_noise={k:v+np.random.rand()*1e-9 for k,v in activity_scores.items()}; pred_act=max(scores_noise,key=scores_noise.get)
        if ground_truth_activity is not None: self.update_knowledge(features,activity_scores,pred_act,ground_truth_activity)
        self.history.append((timestamp,features,activity_scores,pred_act))
        return {'agent_id':self.id,'timestamp':timestamp,'sensor_type':self.sensor_type,'activity_scores':activity_scores,'confidence':self.confidence,'activity_specialization':self.activity_specialization,'prediction':pred_act,'features':features}

    def _extract_features(self, data_chunk):
        features={}; req_cols_present=True
        try:
            if self.sensor_type==SensorType.ACCELEROMETER:
                 req=['x','y','z'];
                 if all(c in data_chunk.columns for c in req): data=data_chunk[req].values; [features.update({n:e(data)}) for n,e in self.feature_extractors.items()]
                 else: req_cols_present=False
            elif self.sensor_type==SensorType.PIR:
                 req=SPHERE_LOCATIONS;
                 if all(c in data_chunk.columns for c in req): data=data_chunk[req]; [features.update({n:e(data)}) for n,e in self.feature_extractors.items()]
                 else: req_cols_present=False
            elif self.sensor_type==SensorType.RSSI:
                 cols=[c for c in data_chunk.columns if c.startswith('AP') or 'rssi' in c.lower()];
                 if cols and all(c in data_chunk.columns for c in cols): data=data_chunk[cols].apply(pd.to_numeric,errors='coerce').values; self.feature_extractors=self._update_num_aps(data.shape[1]); [features.update({n:e(data)}) for n,e in self.feature_extractors.items()]
                 else: req_cols_present=False
            elif self.sensor_type==SensorType.VIDEO:
                 req=['centre_2d_x','centre_2d_y','bb_2d_br_x','bb_2d_br_y','bb_2d_tl_x','bb_2d_tl_y'];
                 if all(c in data_chunk.columns for c in req): data=data_chunk[req].apply(pd.to_numeric,errors='coerce'); [features.update({n:e(data)}) for n,e in self.feature_extractors.items()]
                 else: req_cols_present=False
            if not req_cols_present: return {}
            flat_features={}
            for n,v in features.items():
                if isinstance(v,(np.ndarray,list)):
                    valid=[val for val in (v[~np.isnan(v)] if isinstance(v,np.ndarray) else [i for i in v if pd.notna(i)])]
                    if len(valid)>0:
                        if len(v)>1: [flat_features.update({f"{n}_{i}":val if pd.notna(val) else 0}) for i,val in enumerate(v)]
                        elif len(v)==1: flat_features[n]=v[0] if pd.notna(v[0]) else 0
                elif pd.notna(v): flat_features[n]=v
            return flat_features
        except Exception as e: print(f"Error extracting features for Agent {self.id} ({self.sensor_type}): {e}"); traceback.print_exc(); return {}

    def _update_num_aps(self, num_found):
        if self.sensor_type==SensorType.RSSI:
             num_aps=num_found;
             try: cur_dim=len(self.feature_extractors['mean'](np.array([[]])))
             except: cur_dim=-1
             if num_aps!=cur_dim and num_aps>0:
                 print(f"  Agent {self.id}: Updating RSSI feature extractors for {num_aps} APs.")
                 return {'mean':lambda x:np.nanmean(x,axis=0) if x.shape[0]>0 else np.full(num_aps,-100.0),'std':lambda x:np.nanstd(x,axis=0) if x.shape[0]>0 else np.zeros(num_aps),
                         'max':lambda x:np.nanmax(x,axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>0 else np.full(num_aps,-100.0),'min':lambda x:np.nanmin(x,axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>0 else np.full(num_aps,-100.0),
                         'diff':lambda x:np.nanmax(x,axis=0)-np.nanmin(x,axis=0) if x.shape[0]>0 and np.sum(~np.isnan(x))>1 else np.zeros(num_aps),'stability':lambda x:1.0/(1.0+np.nanstd(x,axis=0)) if x.shape[0]>0 else np.zeros(num_aps)}
        return self.feature_extractors

    def _determine_activity_scores(self, features):
        if not features: return {act:1.0/len(SPHERE_ACTIVITY_NAMES) for act in SPHERE_ACTIVITY_NAMES}
        scores=self.knowledge.copy(); base_mult=1.0
        try:
            if self.sensor_type==SensorType.ACCELEROMETER:
                std_s=features.get('std_0',0)+features.get('std_1',0)+features.get('std_2',0); en_s=features.get('energy_0',0)+features.get('energy_1',0)+features.get('energy_2',0)
                if std_s>1.5 or en_s>1.0:
                    for act,type in SPHERE_ACTIVITIES.items():
                        if type==ActivityType.AMBULATION: scores[act]*=(base_mult*1.5)
                        elif type==ActivityType.TRANSITION: scores[act]*=(base_mult*1.2)
                        elif type==ActivityType.POSTURE: scores[act]*=(base_mult*0.5)
                    if features.get('fft_peak_freq_2',0)>1.5 and features.get('range_2',0)>5.0: scores['a_jump']*=1.5
                    elif features.get('std_0',0)>1.0 or features.get('std_1',0)>1.0: scores['a_walk']*=1.2; scores['a_loadwalk']*=1.2
                elif std_s<0.3 and en_s<0.2:
                     for act,type in SPHERE_ACTIVITIES.items():
                        if type==ActivityType.POSTURE: scores[act]*=(base_mult*1.5)
                        elif type==ActivityType.AMBULATION: scores[act]*=(base_mult*0.5)
                     mz=features.get('mean_2',0); my=features.get('mean_1',0); mx=features.get('mean_0',0); mag=np.sqrt(mx**2+my**2+mz**2); nx,ny,nz=(mx/mag,my/mag,mz/mag) if mag>1e-6 else (0,0,0)
                     if nz>0.8: scores['p_stand']*=1.4
                     elif abs(nx)>0.7 or abs(ny)>0.7: scores['p_lie']*=1.4
                     else: scores['p_sit']*=1.3
                else:
                    for act,type in SPHERE_ACTIVITIES.items():
                         if type==ActivityType.TRANSITION: scores[act]*=(base_mult*1.4)
                         elif type==ActivityType.POSTURE: scores[act]*=(base_mult*0.8)
                         elif type==ActivityType.AMBULATION: scores[act]*=(base_mult*0.8)
            elif self.sensor_type==SensorType.PIR:
                tc=sum(features.get(f'count_{i}',0) for i in range(len(SPHERE_LOCATIONS))); na=sum(features.get(f'any_{i}',0) for i in range(len(SPHERE_LOCATIONS)))
                if tc>3 or na>1:
                    for act,type in SPHERE_ACTIVITIES.items():
                         if type==ActivityType.AMBULATION: scores[act]*=(base_mult*1.4)
                         elif type==ActivityType.POSTURE: scores[act]*=(base_mult*0.6)
                elif tc==0:
                     for act,type in SPHERE_ACTIVITIES.items():
                         if type==ActivityType.POSTURE: scores[act]*=(base_mult*1.2)
                         elif type==ActivityType.AMBULATION: scores[act]*=(base_mult*0.8)
            elif self.sensor_type==SensorType.RSSI:
                n_aps=len([k for k in features if k.startswith('std_')]); avg_std=np.mean([features.get(f'std_{i}',100) for i in range(n_aps)]) if n_aps>0 else 100
                if avg_std>3.0:
                     for act,type in SPHERE_ACTIVITIES.items():
                        if type==ActivityType.AMBULATION: scores[act]*=(base_mult*1.3)
                        elif type==ActivityType.POSTURE: scores[act]*=(base_mult*0.7)
                elif avg_std<1.0:
                     for act,type in SPHERE_ACTIVITIES.items():
                        if type==ActivityType.POSTURE: scores[act]*=(base_mult*1.3)
                        elif type==ActivityType.AMBULATION: scores[act]*=(base_mult*0.7)
            elif self.sensor_type==SensorType.VIDEO:
                asp=features.get('aspect_ratio_mean',1.0); mov=features.get('movement_mean',0.0); h=features.get('height_mean',0.0); yc=features.get('y_center_mean',0.0)
                MVH=15.0; MVL=3.0; ASPT=0.6; ASPW=1.3; HT=150; HS=70; YL=300
                if mov>MVH:
                     for act,type in SPHERE_ACTIVITIES.items():
                        if type==ActivityType.AMBULATION: scores[act]*=(base_mult*1.4)
                        elif type==ActivityType.TRANSITION: scores[act]*=(base_mult*1.2)
                        elif type==ActivityType.POSTURE: scores[act]*=(base_mult*0.6)
                elif mov<MVL:
                    for act,type in SPHERE_ACTIVITIES.items():
                        if type==ActivityType.POSTURE: scores[act]*=(base_mult*1.4)
                        elif type==ActivityType.AMBULATION: scores[act]*=(base_mult*0.6)
                    if asp<ASPT and h>HT: scores['p_stand']*=1.5
                    elif asp>ASPW and yc>YL: scores['p_lie']*=1.5
                    elif h<HS and asp>0.8: scores['p_squat']*=1.2; scores['p_kneel']*=1.2
                    else: scores['p_sit']*=1.3; scores['p_bent']*=1.2
        except KeyError as e: print(f"Warning: Agent {self.id} missing key: {e}")
        except Exception as e: print(f"Error scoring Agent {self.id}: {e}"); traceback.print_exc()
        if self.history:
            last_p=self.history[-1][3]
            if last_p is not None and last_p in scores: scores[last_p]*=1.1
        total=sum(scores.values())
        if total>1e-9:
            norm_s={act:max(0,s/total) for act,s in scores.items()}; final_t=sum(norm_s.values())
            if final_t>1e-9: norm_s={act:s/final_t for act,s in norm_s.items()}
            else: norm_s={act:1.0/len(SPHERE_ACTIVITY_NAMES) for act in SPHERE_ACTIVITY_NAMES}
        else: norm_s={act:1.0/len(SPHERE_ACTIVITY_NAMES) for act in SPHERE_ACTIVITY_NAMES}
        return norm_s

    def update_knowledge(self, features, activity_scores, prediction, ground_truth):
        alpha=self.learning_rate
        if prediction==ground_truth:
            self.confidence=(1-alpha)*self.confidence+alpha*1.0; self.activity_accuracy[prediction]=(1-alpha)*self.activity_accuracy[prediction]+alpha*1.0; self.activity_specialization[prediction]=(1-alpha)*self.activity_specialization[prediction]+alpha*1.0
        else:
            self.confidence=(1-alpha)*self.confidence+alpha*0.0
            if prediction is not None: self.activity_accuracy[prediction]=(1-alpha)*self.activity_accuracy[prediction]+alpha*0.0; self.activity_specialization[prediction]=max(0.01,(1-alpha)*self.activity_specialization[prediction]+alpha*0.3)
            self.activity_accuracy[ground_truth]=(1-alpha)*self.activity_accuracy[ground_truth]+alpha*0.3; self.activity_specialization[ground_truth]=max(0.01,(1-alpha)*self.activity_specialization[ground_truth]+alpha*0.4)
        beta=alpha*0.5; cur_k_sum=sum(self.knowledge.values())
        for act,score in activity_scores.items(): self.knowledge[act]=(1-beta)*self.knowledge[act]+beta*score*cur_k_sum
        total_k=sum(self.knowledge.values())
        if total_k>1e-9: self.knowledge={act:k/total_k for act,k in self.knowledge.items()}

# --- SwarmQueen Integration System ---
class SwarmQueen:
    """Manages the swarm of agents and integrates their outputs."""
    def __init__(self, agents, activity_names, location_names):
        self.agents={a.id:a for a in agents}; self.activity_names=activity_names; self.location_names=location_names; self.num_activities=len(activity_names); self.system_history=deque(maxlen=20)
        self.location_activity_prob=self._initialize_location_activity_map(); self.confusion_matrix=pd.DataFrame(np.zeros((self.num_activities,self.num_activities)),index=activity_names,columns=activity_names)
        self.fusion_confidence_weight=1.0; self.fusion_specialization_weight=1.0; self.temporal_inertia_boost=1.1

    def _initialize_location_activity_map(self):
        loc_map={loc:{act:1.0/self.num_activities for act in self.activity_names} for loc in self.location_names+['UNKNOWN']}; base_p=1.0/self.num_activities
        def update(loc,factors):
            if loc not in loc_map: return
            for act,f in factors.items():
                 if act in loc_map[loc]: loc_map[loc][act]*=f
            total=sum(loc_map[loc].values())
            if total>1e-9: loc_map[loc]={act:p/total for act,p in loc_map[loc].items()}
            else: loc_map[loc]={act:base_p for act in self.activity_names}
        update('living',{'p_sit':4,'p_lie':2,'a_walk':1.5,'t_sit_stand':1.5,'t_stand_sit':1.5}); update('bed1',{'p_lie':5,'p_sit':2,'t_lie_sit':2,'t_sit_lie':2}); update('bed2',{'p_lie':5,'p_sit':2,'t_lie_sit':2,'t_sit_lie':2})
        update('kitchen',{'p_stand':3,'a_walk':2,'p_bent':1.5}); update('bath',{'p_stand':3,'a_walk':1.5,'p_bent':1.5}); update('toilet',{'p_sit':5,'p_stand':1.5,'t_sit_stand':1.5,'t_stand_sit':1.5})
        update('stairs',{'a_ascend':6,'a_descend':6,'p_stand':1.5,'a_walk':1.2}); update('hall',{'a_walk':4,'p_stand':2}); update('study',{'p_sit':5,'p_stand':1.5,'t_sit_stand':1.2,'t_stand_sit':1.2})
        loc_map['UNKNOWN']={act:base_p for act in self.activity_names}; return loc_map

    def process_system_reading(self, timestamp, data_chunks_by_sensortype, ground_truth_activity=None, ground_truth_location=None):
        agent_outs={}; agent_locs={}
        for agent_id,agent in self.agents.items():
            chunk=data_chunks_by_sensortype.get(agent.sensor_type,None); gt=ground_truth_activity
            output=agent.process_reading(timestamp,chunk,ground_truth_activity=gt); agent_outs[agent_id]=output
            if output and 'error' not in output:
                if agent.sensor_type==SensorType.PIR and output.get('features'):
                    any_p=[output['features'].get(f'any_{i}',0) for i in range(len(self.location_names))]
                    if sum(any_p)==1: idx=np.argmax(any_p); agent_locs[agent_id]={'location':self.location_names[idx],'confidence':agent.confidence}
        fused=self._weighted_fusion(agent_outs); inferred_loc=self._determine_location(agent_locs,ground_truth_location)
        context=self._apply_location_context(fused,inferred_loc); final_s=self._apply_temporal_consistency(context)
        if not final_s: final_p=None
        else: noise_s={k:v+np.random.rand()*1e-9 for k,v in final_s.items()}; final_p=max(noise_s,key=noise_s.get)
        self.system_history.append((timestamp,final_p,inferred_loc,final_s))
        if ground_truth_activity is not None and final_p is not None: self.update_confusion_matrix(ground_truth_activity,final_p)
        return {'timestamp':timestamp,'final_prediction':final_p,'final_scores':final_s,'inferred_location':inferred_loc,'agent_outputs':agent_outs}

    def _determine_location(self, agent_locations, ground_truth_location=None):
         if not agent_locations: return "UNKNOWN"
         votes=defaultdict(float); total_conf=0.0
         for agent_id,info in agent_locations.items():
             conf=info.get('confidence',0.1); loc=info.get('location','UNKNOWN')
             if loc!='UNKNOWN': votes[loc]+=conf; total_conf+=conf
         if not votes or total_conf<0.1:
            if self.system_history: last_loc=self.system_history[-1][2]; return last_loc if last_loc!="UNKNOWN" else "UNKNOWN"
            return "UNKNOWN"
         return max(votes,key=votes.get)

    def _weighted_fusion(self, agent_outputs):
        fused={act:0.0 for act in self.activity_names}; total_w={act:1e-9 for act in self.activity_names}
        for agent_id,output in agent_outputs.items():
            if not output or 'error' in output or not output.get('activity_scores'): continue
            conf=output.get('confidence',0.1); spec=output.get('activity_specialization',{act:0.5 for act in self.activity_names}); scores=output['activity_scores']
            for act,score in scores.items():
                if act not in fused: continue
                ct=max(0.01,conf)**self.fusion_confidence_weight; st=max(0.01,spec.get(act,0.5))**self.fusion_specialization_weight; w=ct*st
                fused[act]+=score*w; total_w[act]+=w
        for act in fused: fused[act]/=total_w[act]
        final_t=sum(fused.values())
        if final_t>1e-9: fused={act:s/final_t for act,s in fused.items()}
        else: fused={act:1.0/self.num_activities for act in self.activity_names}
        return fused

    def _apply_location_context(self, scores, location):
        if location not in self.location_activity_prob: location="UNKNOWN"
        context={}; probs=self.location_activity_prob[location]
        for act,score in scores.items(): prior=probs.get(act,1.0/self.num_activities); context[act]=score*prior
        total=sum(context.values())
        if total>1e-9: context={act:s/total for act,s in context.items()}
        else: context={act:1.0/self.num_activities for act in self.activity_names}
        return context

    def _apply_temporal_consistency(self, current_scores):
        if not self.system_history: return current_scores
        last_p=self.system_history[-1][1]; smoothed=current_scores.copy()
        if last_p is not None and last_p in smoothed: smoothed[last_p]*=self.temporal_inertia_boost
        total=sum(smoothed.values())
        if total>1e-9: smoothed={act:s/total for act,s in smoothed.items()}
        else: smoothed={act:1.0/self.num_activities for act in self.activity_names}
        return smoothed

    def update_confusion_matrix(self, gt, pred):
        if gt in self.confusion_matrix.index and pred in self.confusion_matrix.columns: self.confusion_matrix.loc[gt,pred]+=1

    def get_confusion_matrix(self): return self.confusion_matrix

# --- Main Execution Logic ---
if __name__ == "__main__":
    print("Initializing SPHERE Swarm Intelligence System...")
    # San Diego Time: Monday, March 31, 2025 at 4:43 AM PDT
    print(f"Current Time: {time.strftime('%Y-%m-%d %H:%M:%S')}")

    # --- Configuration ---
    DATA_PATH = "/content/SPHERE_unzipped" # <<< --- VERIFY YOUR UNZIPPED DATA PATH --- <<<
    TARGET_SUBJECT = "00001" # <<<--- USE CORRECT SUBJECT ID (e.g., "00001") ---<<<
    WINDOW_SIZE_SEC = 2.0
    OVERLAP_SEC = 1.0
    RESAMPLE_FREQ = '50ms' # Corresponds to 20 Hz

    # --- Initialize Agents ---
    agents_to_use = [
        Agent(agent_id="Accel_01", sensor_type=SensorType.ACCELEROMETER, learning_rate=0.05, history_len=10),
        Agent(agent_id="PIR_01", sensor_type=SensorType.PIR, learning_rate=0.1, history_len=5),
    ]
    print(f"Initialized {len(agents_to_use)} agents.")

    # --- Initialize SwarmQueen ---
    queen = SwarmQueen(agents_to_use, SPHERE_ACTIVITY_NAMES, SPHERE_LOCATIONS)
    print("Initialized SwarmQueen.")

    # --- Data Loading ---
    print("\n--- Starting Data Loading and Processing ---")
    print(f"Target Subject: {TARGET_SUBJECT}")
    print(f"Attempting to load data from: {DATA_PATH}")
    accel_data = load_sphere_data(DATA_PATH, TARGET_SUBJECT, 'accelerometer')
    pir_data = load_sphere_data(DATA_PATH, TARGET_SUBJECT, 'pir')
    annotations = load_sphere_data(DATA_PATH, TARGET_SUBJECT, 'annotations')

    # --- Robustness Check ---
    if accel_data is None or annotations is None:
         print("\n--- ERROR: Failed to load essential Accelerometer or Annotation data. Stopping. ---")
    else:
        print("\n--- Essential data (Accel, Annotations) loaded successfully. Proceeding... ---")
        if pir_data is None: print("--- Warning: Failed to load PIR data correctly. Proceeding without it. ---")
        else: print("--- PIR data loaded successfully. ---")

        all_data_streams = { SensorType.ACCELEROMETER: accel_data, SensorType.PIR: pir_data }
        sync_data = synchronize_data(all_data_streams, resample_freq=RESAMPLE_FREQ)

        if sync_data is None or sync_data.empty: print("\n--- ERROR: Data synchronization failed. Stopping. ---")
        else:
            window_generator = create_time_windows(sync_data, WINDOW_SIZE_SEC, OVERLAP_SEC)
            print("Sorting annotations index...")
            if 'end' not in annotations.columns: print("Error: Annotations missing 'end' column."); exit()
            annotations = annotations.sort_index()

            # --- Processing Loop ---
            print("\n--- Processing Windows ---")
            all_predictions=[]; all_ground_truths=[]; window_count=0; start_processing_time=time.time()
            debug_gt_lookup = True # Keep debugging prints enabled for now
            # ------------------------------------
            for window in window_generator:
                window_count+=1; window_end_time=window['timestamp']; window_data_chunks=window['data']
                ground_truth=None
                try: # Ground Truth Lookup
                    window_start_time=window_end_time-pd.Timedelta(seconds=WINDOW_SIZE_SEC); window_mid_point=window_start_time+(window_end_time-window_start_time)/2

                    # --- DEBUG PRINTS START ---
                    if debug_gt_lookup and (window_count <= 5 or window_count % 100 == 1):
                         print(f"\n  [DEBUG GT W{window_count}] Window: [{window_start_time}, {window_end_time})")
                         print(f"  [DEBUG GT W{window_count}] Midpoint: {window_mid_point}")
                    # --- DEBUG PRINTS END ---

                    match_condition = (annotations.index <= window_mid_point) & (annotations['end'] > window_mid_point)
                    matching_annotations = annotations.loc[match_condition]

                    if debug_gt_lookup and (window_count <= 5 or window_count % 100 == 1):
                         print(f"  [DEBUG GT W{window_count}] Match Condition Sum: {match_condition.sum()}")

                    if not matching_annotations.empty:
                        matched_annotation = matching_annotations.iloc[0]

                        if debug_gt_lookup and (window_count <= 5 or window_count % 100 == 1):
                             print(f"  [DEBUG GT W{window_count}] Matched Annotation Row (Index={matched_annotation.name}):\n{matched_annotation.to_string()}")
                             print(f"  [DEBUG GT W{window_count}] Available columns in annotations df: {annotations.columns.tolist()}")

                        # --- <<< FIX: Add 'name' to the list of possible activity columns >>> ---
                        activity_col = next((col for col in ['activity', 'annotation', 'label', 'name'] if col in annotations.columns), None) # Check for 'name' now

                        if debug_gt_lookup and (window_count <= 5 or window_count % 100 == 1):
                             print(f"  [DEBUG GT W{window_count}] Activity Col Found?: {activity_col}") # Should now find 'name'

                        if activity_col:
                            ground_truth = matched_annotation[activity_col]
                            if debug_gt_lookup and (window_count <= 5 or window_count % 100 == 1):
                                 print(f"  [DEBUG GT W{window_count}] Ground Truth Set To: '{ground_truth}'") # Should show the activity
                        # else: # This case should be less likely now
                            # print(f"  [DEBUG GT W{window_count}] Could not find activity column")

                except Exception as e:
                    print(f"Error looking up annotation for window ending at {window_end_time} (midpoint {window_mid_point}): {e}")
                    traceback.print_exc()
                # --- End Ground Truth Lookup ---

                system_output=queen.process_system_reading(window_end_time,window_data_chunks,ground_truth_activity=ground_truth)
                if ground_truth is not None and ground_truth in SPHERE_ACTIVITY_NAMES: all_ground_truths.append(ground_truth); all_predictions.append(system_output['final_prediction'])
                # Print normal progress (less frequently now maybe)
                if window_count % 200 == 0: print(f"  Processed {window_count} windows... Time: {window_end_time.strftime('%H:%M:%S.%f')[:-3]} Pred: {system_output['final_prediction']} (GT: {ground_truth if ground_truth else 'N/A'}) Loc: {system_output['inferred_location']}")
            # --- End Window Processing Loop ---

            end_processing_time=time.time(); print(f"\n--- Finished processing {window_count} windows in {end_processing_time-start_processing_time:.2f} seconds ---")

            # --- Evaluation ---
            print("\n--- Evaluation Results ---"); final_confusion_matrix=queen.get_confusion_matrix(); print(f"Collected {len(all_ground_truths)} predictions with valid ground truth.")
            if len(all_ground_truths) > 0 and len(all_predictions) == len(all_ground_truths):
                accuracy=accuracy_score(all_ground_truths,all_predictions); print(f"\nOverall Accuracy: {accuracy:.4f}")
                unique_labels=sorted(list(set(all_ground_truths)|set(all_predictions))); unique_labels=[label for label in unique_labels if label is not None and label in SPHERE_ACTIVITY_NAMES]
                print("\nClassification Report:");
                try: report=classification_report(all_ground_truths,all_predictions,labels=unique_labels,zero_division=0); print(report)
                except ValueError as e: print(f"Could not generate classification report: {e}")
                print("\nFinal Confusion Matrix:"); cm_labels=sorted(list(set(all_ground_truths)|set(all_predictions))); cm_labels=[l for l in cm_labels if l is not None and l in final_confusion_matrix.index and l in final_confusion_matrix.columns]
                if cm_labels:
                    cm_display=final_confusion_matrix.loc[cm_labels,cm_labels]; print(cm_display)
                    try:
                        plt.figure(figsize=(12,10)); sns.heatmap(cm_display,annot=True,fmt=".0f",cmap="Blues",xticklabels=cm_labels,yticklabels=cm_labels); plt.title(f'System Confusion Matrix (Subject: {TARGET_SUBJECT})')
                        plt.ylabel('True Label'); plt.xlabel('Predicted Label'); plt.xticks(rotation=45,ha='right'); plt.yticks(rotation=0); plt.tight_layout()
                        plot_filename=f"confusion_matrix_{TARGET_SUBJECT}.png"; plt.savefig(plot_filename); print(f"\nConfusion matrix plot saved as {plot_filename}"); plt.close()
                    except Exception as plot_err: print(f"Error plotting confusion matrix: {plot_err}")
                else: print("\nCould not generate confusion matrix (no common valid labels found).")
            else:
                 if len(all_ground_truths)==0: print("\nEvaluation failed: No valid ground truth labels were found."); print("Check annotation loading and GT lookup logic (including timestamps/units and activity column name).")
                 else: print("\nEvaluation failed: Length mismatch between predictions and ground truth lists.")

    print("\nSystem processing finished (or terminated due to loading/sync error).")



Initializing SPHERE Swarm Intelligence System...
Current Time: 2025-03-31 11:44:09
Initialized 2 agents.
Initialized SwarmQueen.

--- Starting Data Loading and Processing ---
Target Subject: 00001
Attempting to load data from: /content/SPHERE_unzipped
Attempting to load accelerometer for subject 00001 from base path: /content/SPHERE_unzipped
  Using subject data folder: /content/SPHERE_unzipped/train/00001
  Loading data from: /content/SPHERE_unzipped/train/00001/acceleration.csv
  Found timestamp column: 't'
  Converting timestamp column (assuming units are seconds)...
  ...Loaded 35710 rows.
Attempting to load pir for subject 00001 from base path: /content/SPHERE_unzipped
  Using subject data folder: /content/SPHERE_unzipped/train/00001
  Loading data from: /content/SPHERE_unzipped/train/00001/pir.csv
  Using 'start' column as timestamp index for pir.
  ...Processed 115 pir rows.
Attempting to load annotations for subject 00001 from base path: /content/SPHERE_unzipped
  Using subject